In [1]:
from bs4 import BeautifulSoup
import requests as rq
from unicodedata import normalize
import re

In [2]:
url = "https://www.businessinsider.com/author/francis-agustin"

In [3]:
response = rq.get(url)
page_source = BeautifulSoup(response.text, 'lxml')

In [4]:
pos = page_source.find('p', attrs={'data-test': "author-aside-title"}).text.strip()
pos

'Business News Fellow'

In [17]:
author_bio = [a.text.strip() for a in page_source.select('div.col-12.col-md-8.author-description > p')]
author_bio = normalize("NFKD", ''.join(map(str, author_bio)).replace('  ', ' ').strip())
author_bio

"Francis is a Weekend Fellow on Business Insider's News Desk. He is currently based in California.He is a multimedia journalist who has covered topics from corporate news to politics, to youth trends, digital culture/technology, and crime. He attended the University of Southern California, where he studied communication and journalism, and served as a news editor on multiple desks.Have a news tip? Email Francis from a non-work email at fagustin@insider.com"

In [21]:
# 'a.author-contact-icon-link.share-link.email::attr(href)'

author_email = page_source.find('a', attrs={'class': 'author-contact-icon-link share-link email'})['href'].replace('mailto:', '').strip()
author_email

'fagustin@insider.com'

In [22]:
author_twitter = page_source.find('a', attrs={
                    'class': 'author-contact-icon-link share-link twitter'})['href'].strip()
author_twitter

'https://twitter.com/francisnagustin'

In [2]:
url = 'https://markets.businessinsider.com/news/currencies/stablecoin-regulation-cryptocurrency-banks-biden-treasury-bitcoin-circle-tether-binance-2021-10'

In [3]:
response = rq.get(url)
page_source = BeautifulSoup(response.text, 'lxml')

In [30]:
def remove_articles(text):
    # strip the unicode articles
    #text = normalize("NFKD", text.strip(u'\u201c'u'\u201d'))
    text = normalize("NFKD", ''.join(map(str, text)).replace('  ', ' ').strip())
    return text

from itemloaders.processors import Join, MapCompose
from scrapy.item import Item, Field

In [35]:
proc = MapCompose(remove_articles)

class InsiderArtItem(Item):
    content = Field(
        input_processor=MapCompose(remove_articles),
        # TakeFirst return the first value not the whole list
        output_processor=Join()
        )

article = InsiderArtItem()

In [32]:
article_content = [a.text.strip() for a in page_source.select(
    'div.col-xs-12.news-content.no-padding > p, div.col-xs-12.news-content.no-padding > ul > li[aria-level*="1"]')] # div.col-xs-12.news-content.no-padding > p > a,
article_content

['The Biden administration is looking into regulating stablecoin issuers as if they are banks - including nudging these firms to register as banks - as the rapidly evolving cryptocurrency space continues to elicit concern from authorities, The Wall Street Journal first reported Friday.',
 'The White House is also considering asking Congress to propose legislation to create a special-purpose charter especially tailored to these firms, sources told The Journal. A special-purpose charter would subject the companies to federal oversight rather than state-by-state laws.',
 "The administration's recommendations, though still a work in progress, will be included in a Treasury-led paper slated for publishing in late October. Among the authors of the paper are Treasury Secretary Janet Yellen, Federal Reserve Chairman Jerome Powell, and Securities and Exchange Commission Chairman Gary Gensler, who recently compared stablecoins to poker chips.",
 'One recommendation is having the Financial Stabil

In [36]:
# article['content'] = article_content
# article['content']

# article['content'] = [remove_articles(line) for line in article_content]
# article['content'] = ' '.join(article['content'])
# article['content']

article_content = [remove_articles(a.text) for a in page_source.select(
    'div.col-xs-12.news-content.no-padding > p, div.col-xs-12.news-content.no-padding > ul > li[aria-level*="1"]')] # div.col-xs-12.news-content.no-padding > p > a,
article_content = ''.join(article_content)
article_content

'The Biden administration is looking into regulating stablecoin issuers as if they are banks - including nudging these firms to register as banks - as the rapidly evolving cryptocurrency space continues to elicit concern from authorities, The Wall Street Journal first reported Friday.The White House is also considering asking Congress to propose legislation to create a special-purpose charter especially tailored to these firms, sources told The Journal. A special-purpose charter would subject the companies to federal oversight rather than state-by-state laws.The administration\'s recommendations, though still a work in progress, will be included in a Treasury-led paper slated for publishing in late October. Among the authors of the paper are Treasury Secretary Janet Yellen, Federal Reserve Chairman Jerome Powell, and Securities and Exchange Commission Chairman Gary Gensler, who recently compared stablecoins to poker chips.One recommendation is having the Financial Stability Oversight C